In [1]:
import os
import numpy as np
import pandas as pd
from random import random

import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable

from gmplot import GoogleMapPlotter
import geopandas as gpd
from fiona.drvsupport import supported_drivers
supported_drivers['KML'] = 'rw'

import webbrowser
import csv


In [2]:
# Rutas
ROOT_PATH = 'C:/Users/pedro/ds-geo'
CONFIG_PATH = ROOT_PATH + '/config'
MAPS_PATH = ROOT_PATH + '/maps'
PARCELAS_PATH = ROOT_PATH + '/data'

os.chdir(ROOT_PATH)


## GoogleMaps

In [3]:
class CustomGoogleMapPlotter(GoogleMapPlotter):
    
    # __init__
    def __init__(self, center_lat, center_lng, zoom, apikey='',
                 map_type='satellite'):
        if apikey == '':
            try:
                with open(CONFIG_PATH + '/apikey.txt', 'r') as apifile:
                    apikey = apifile.readline()
            except FileNotFoundError:
                pass
        super().__init__(center_lat, center_lng, zoom, apikey=apikey)

        self.map_type = map_type
        assert(self.map_type in ['roadmap', 'satellite', 'hybrid', 'terrain'])
         

    # Añade polígonos y parcelas al mapa
    def AddMapPolygons(self, df, mode=None):

        for index, row in df.iterrows():
            l_lat = row['Latitude']
            l_long = row['Longitude']

            if mode == 'C':
                s_color = row['ColorCultivo']
                if s_color == '':
                    s_color = 'white'
            elif mode == 'P':
                s_color = row['ColorPropietario']
                if s_color == '':
                    s_color = 'white'
            else:
                s_color = 'cornflowerblue'

            self.polygon(l_lat, l_long, color=s_color, face_alpha=0.5) 
   

    # Añade puntos al mapa
    def AddMapPoints(self, df, mode=None):

        for index, row in df.iterrows():
            l_lat = row['Latitude']
            l_long = row['Longitude']

            if mode == 'C':
                s_color = row['ColorCultivo']
                if s_color == '':
                    s_color = 'white'
            elif mode == 'P':
                s_color = 'white'
            else:
                s_color = 'cornflowerblue'

            self.circle(l_lat, l_long, radius=10, alpha=0.5, color=s_color)

            

## Carga de ficheros

In [4]:
# Carga puntos de elementos destacados

def LoadPoints():
    df_temp = pd.read_csv(PARCELAS_PATH + '/points.csv', sep=',', encoding='UTF-8')
    df_info = pd.read_csv(CONFIG_PATH + '/cultivos.info', sep=',', encoding='UTF-8')
    df_points = pd.merge(df_temp, df_info[['CodigoCultivo', 'ColorCultivo']], left_on=['Tipo'],  right_on=['CodigoCultivo'], how='left')
    df_points.fillna('', inplace=True)
    df_points.drop('CodigoCultivo', axis=1, inplace=True)
    df_points.rename(columns={'Latitud':'Latitude', 'Longitud':'Longitude'}, inplace=True)
    
    return df_points
    

In [5]:
# Carga información de las parcelas

def LoadParcelasInfo():
    
    df_infoA = pd.read_csv(PARCELAS_PATH + '/parcelas.csv', sep=',', encoding='UTF-8', na_filter= False)
    df_infoC = pd.read_csv(CONFIG_PATH + '/cultivos.info', sep=',', encoding='UTF-8', na_filter= False)
    df_infoP = pd.read_csv(CONFIG_PATH + '/propietarios.info', sep=',', encoding='UTF-8', na_filter= False)
    
    df_info = pd.merge(df_infoA, df_infoC, on=['CodigoCultivo'], how='left')
    df_info = pd.merge(df_info, df_infoP, on=['CodigoPropietario'], how='left')
    
    df_info.fillna('', inplace=True)
    df_info.drop(['CodigoCultivo', 'CodigoPropietario'], axis=1, inplace=True)
    
    return df_info
    

In [6]:
# Carga de todos los ficheros del catastro, en formato KML.

def LoadCatastroKML(s_file):
    file_polygons = gpd.read_file(PARCELAS_PATH + '/' + s_file + '.kml', driver='KML')
    df_parc = pd.DataFrame(columns=['Ref', 'Sub', 'Latitude', 'Longitude'])
    
    for index, row in file_polygons.iterrows():
        l_lat = []
        l_long = []
        if len(row['Name']) > 0:
            s_sub = row['Name']
        else:
            s_sub = 'z'
        
        for point in list(row['geometry'].exterior.coords): 
            l_lat.append(point[1])
            l_long.append(point[0])

        row_new = {'Ref':s_file, 'Sub':s_sub, 'Latitude':l_lat, 'Longitude':l_long}
        df_new = pd.DataFrame([row_new])

        df_parc = pd.concat([df_parc, df_new], axis=0, ignore_index=True)
    
    return df_parc


def LoadCatastro():

    files_list = []
    for file in os.listdir(PARCELAS_PATH):
        if file.endswith(".kml"):
            files_list.append(file.replace('.kml', ''))

    df_parcelas = pd.DataFrame(columns=['Ref', 'Sub', 'Latitude', 'Longitude'])

    for file in files_list:
        df_new = LoadCatastroKML(file)
        df_parcelas = pd.concat([df_parcelas, df_new], axis=0, ignore_index=True)

        
    # Carga de la información complementaria de las parcelas
    df_infoParcelas = LoadParcelasInfo()

    # Completamos la información de las parcelas
    df_compl = pd.merge(df_parcelas, df_infoParcelas, on=['Ref', 'Sub'], how='left')
    df_compl.fillna('', inplace=True)

    return df_compl


## Otras funciones

In [7]:
# Cálculo del centro del mapa y del zoom inicial (para los ficheros de parcelas KML)

def GetMapInitialSizeKML(df):
    
    max_lat = df['Latitude'].apply(max).max()
    min_lat = df['Latitude'].apply(min).min()
    max_long = df['Longitude'].apply(max).max()
    min_long = df['Longitude'].apply(min).min()
    
    center_lat = min_lat + (max_lat - min_lat)/2
    center_long = min_long + (max_long - min_long)/2
    
    width_lat = abs(max_lat - min_lat)
    width_long = abs(max_long - min_long)
    
    zoom = np.ceil(1.3 * (1 / max(width_lat, width_long)))
    zoom = max(zoom, 13)
    zoom = min(zoom, 17)
    
    return (center_lat, center_long, zoom)



In [8]:
# Generación del mapa con los elementos de la entrada
# Se apoya en la clase CustomGoogleMapPlotter, y a su vez en GoogleMapPlotter
# mode: 'C' = cultivo, 'P' = Propietario
# initial: '' = posición y zoom por defecto, '' = cálculo automático

def GenerateMapComp(polygons, points=None, markers=None, mode='', initial=''):

    if initial == 'auto':
        initial_lat, initial_long, initial_zoom = GetMapInitialSizeKML(polygons)
        print(initial_lat, initial_long, initial_zoom)
    else:
        initial_lat = 39.7013042
        initial_long = -2.9017362 
        initial_zoom = 13
    
    s_mode = mode.upper()
    
    gmap = CustomGoogleMapPlotter(initial_lat, initial_long, initial_zoom, map_type='satellite')
    
    if polygons is not None:
        gmap.AddMapPolygons(polygons, s_mode)    
        
    if points is not None:
        gmap.AddMapPoints(points, s_mode)
    
    if s_mode == 'C':
        s_mapname = 'mapa_cultivos.html'
    elif s_mode == 'P':
        s_mapname = 'mapa_propietarios.html'
    else:
        s_mapname = 'mapa.html'

    gmap.draw(MAPS_PATH + '/' + s_mapname)    

    # Abrir mapa en el navegador
    url = "file://" + MAPS_PATH + '/' + s_mapname
    webbrowser.open_new_tab(url)


### Ejecución

In [9]:
# Carga de las parcelas
df_parcelas = LoadCatastro()

# Carga puntos destacados
df_points = LoadPoints()

# Genera el mapa y lo abre en el navegador
GenerateMapComp(polygons=df_parcelas, points=df_points, mode='p')
